In [1]:
!pip install transformers datasets evaluate sentencepiece -q
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf' -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done


In [ ]:
import torch
import evaluate
import numpy as np

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [ ]:
dataset = load_dataset("nsmc")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset['train'] = dataset['train'].filter(lambda x: len(x['document']) >= 20)
dataset['test'] = dataset['test'].filter(lambda x: len(x['document']) >= 20)

In [ ]:
small_train_dataset = dataset["train"].shuffle(seed=42).select(range(3000))
small_eval_dataset = dataset["test"].shuffle(seed=42).select(range(100))

In [ ]:
small_eval_dataset.to_json("data.json")

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

28396

In [ ]:
!head data.json

{"id":"6441922","document":"3\uc810\ub3c4 \uc904\uae4c\ub9d0\uae4c\ud588\ub294\ub370 \ud655\uc2e4\ud788 \ucda4\uc5d0\uc784\ud329\ud2b8\ub3c4\uc5c6\uace0 \uc2ec\uc2ec\ud55c\ub290\ub08c\ub9cc\uc788\uace0 \uc660\uc9c0\ubaa8\ub974\uac8c \uc544\ub9c8\ucd94\uc5b4\ub4e4\uc774\ud558\ub294\ub290\ub08c\ub9cc \ub4e4\uc5c8\ub2e4 \uc2a4\ud1a0\ub9ac\ub3c4 \uc2a4\ud15d\uc5c52:\ub354\uc2a4\ud2b8\ub9ac\ud2b8\ubc30\uaf08\ub294\ub370 \ubc30\ub080\uac70\uc5d0\ub3c4\ubd88\uad6c\ud558\uace0 \uc2a4\ud1a0\ub9ac\uac00 \ub354\uc774\uc0c1\ud574\uc9d0 \uc560\ub4e4\ub178\ub294\uac83\uac19\uc74c \ud3c9\uc81010\uc810\uc900\ubd84\ub4e4\uc911\uc5d0\uc11c\uc2a4\ud15d\uc5c5 \uc548\ubcf4\uc2e0\ubd84\uc2a4\ud15d\uc5c5\ud55c\ubc88\ubcf4\uba74\uc774\uc601\ud654\uc4f0\ub808\uae30\uac19\uc74c","label":0}
{"id":"8932624","document":"\uc870\ud3ed\uacfc \uc5ec\uc758\uc0ac\uc758 \ud669\ub2f9 \ub7ec\ube0c \uc2a4\ud1a0\ub9ac. \ud55c\uad6d \uba5c\ub85c \uc601\ud654\uc758 \ub098\uc05c \uc810.","label":0}
{"id":"10011962","document":"

In [ ]:
sample_dataset_for_loading = load_dataset("json", data_files={"test": "data.json"})
sample_dataset_for_loading

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 100
    })
})

In [ ]:
# tokenized dataset을 torch에서 사용 가능한 포맷으로 처리
train_tensor = tokenized_train_dataset.remove_columns(["id", "document"])
train_tensor = train_tensor.rename_column("label", "labels")
train_tensor.set_format("torch")

eval_tensor = tokenized_eval_dataset.remove_columns(["id", "document"])
eval_tensor = eval_tensor.rename_column("label", "labels")
eval_tensor.set_format("torch")

In [ ]:
small_train_dataset[100]

{'id': '8543349',
 'document': '러닝타임의 123분..?4편을 편집해서 나오나보군요..TV드라마 4부작인것으로 알고 있었는데..정말 집중하면서 보게되고 , 독일의 또 다른 관점의 전쟁의 실상 영화라 할수 있죠. 정말 추천드리고 싶은 영화?(드라마입니다)',
 'label': 1}

In [ ]:
# KoBERT Tokenizer 라이브러리 사용
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

def tokenize_function(examples):
    return tokenizer(examples["document"], padding="max_length", max_length=512, truncation=True)

# tokenization
tokenized_train_dataset = small_train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = small_eval_dataset.map(tokenize_function, batched=True)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# 모델 설정 및 학습 설정
model = AutoModelForSequenceClassification.from_pretrained("skt/kobert-base-v1", num_labels=2)
training_args = TrainingArguments(
    num_train_epochs=3.0,
    output_dir="output",
    do_train=True,
    do_eval=False,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 모델 성능 평가
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Trainer 라이브러리 사용
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
# 학습 전 성능
trainer.evaluate()

{'eval_loss': 0.698937177658081,
 'eval_accuracy': 0.47,
 'eval_runtime': 0.9195,
 'eval_samples_per_second': 108.751,
 'eval_steps_per_second': 14.138}

In [ ]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.662900,0.485129,0.790000
2,0.607500,0.531183,0.760000
3,0.342700,0.488285,0.790000


TrainOutput(global_step=1125, training_loss=0.5691568569607205, metrics={'train_runtime': 233.1567, 'train_samples_per_second': 38.601, 'train_steps_per_second': 4.825, 'total_flos': 2367999498240000.0, 'train_loss': 0.5691568569607205, 'epoch': 3.0})

In [ ]:
# 학습 후 성능
trainer.evaluate()

{'eval_loss': 0.48828476667404175,
 'eval_accuracy': 0.79,
 'eval_runtime': 0.8927,
 'eval_samples_per_second': 112.017,
 'eval_steps_per_second': 14.562,
 'epoch': 3.0}

In [ ]:
from transformers import pipeline

classifier = pipeline(model=model, tokenizer=tokenizer, task="text-classification", device="cuda")

In [ ]:
print(classifier("끝내주는 영화였음. 꼭 다시봐야지. 온가족 데리고 와야겠다. 강추 꼭 보셈!!"))

[{'label': 'LABEL_1', 'score': 0.8739438056945801}]


In [ ]:
print(classifier("별 그지같은 영화를 다보겠네!! 완전 돈버렸음. 내가 이래서 한국영화 안본다!"))

[{'label': 'LABEL_1', 'score': 0.654887855052948}]
